<h1 style="text-align: center;">Computational Theory Tasks</h1>

This notebook contains explanations and solutions for 8 computational theory tasks. Each section begins with a description of the task, outlining its origin and signifigance. Then the solution is explained, followed by code cells that implement the solution. Finally, code cells containing a suite of tests designed to validate the solution follow.

To supplement the notebook, the following sidebars can be found after each solution:

<div style="display: flex; justify-content: center;">
  <div style="display: flex; flex-direction: column; gap: 12px; max-width: 800px; width: 100%;">

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #5C9E75; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Test Overview</strong>
      </div>
      <div style="flex: 1; border: 3px solid #5C9E75; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Explains the tests used to validate each solution, covering the inputs, expected results, and the rationale.
      </div>
</div>

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #4F83CC; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Sources</strong>
      </div>
      <div style="flex: 1; border: 3px solid #4F83CC; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Links to the project's sources, along with concise descriptions of how they influenced development.
      </div>
</div>

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #6C3BAA; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Alternatives</strong>
      </div>
      <div style="flex: 1; border: 3px solid #6C3BAA; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Compares the solution against other potential approaches, providing a justification for the final choice.
      </div>
</div>

  </div>
</div>


### Imports

In [21]:
# Imports
import hashlib
import os
import struct
import unittest
import tempfile

### Task 1 - Binary Representations

This task involves implementing several binary manipulation functions. Bitwise operations are computational techniques that work directly with individual bits in a binary number, including operations like AND, OR, NOT, and rotation. They operate at the binary level to manipulate individual bits more efficiently than higher-level arithmetic functions [(1)](https://www.techtarget.com/whatis/definition/bitwise). In cybersecurity, these operations form the backbone of hash functions like SHA-256 by producing fixed-size outputs that are "virtually impossible to transform back into their original data" [(2)](https://brilliant.org/wiki/secure-hashing-algorithms/). These operations began appearing in cryptography in the late 1970s and 1980s [(3)](https://link.springer.com/chapter/10.1007/978-3-642-11925-5_1), and became standardized components of important security protocols including SSL, TLS, SSH, and IPsec, which are crucial for securing digital communications across the internet.

<div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 900px;">

**Task 1 Background Sources**
- **(1)** [TechTarget's "What is Bitwise?"](https://www.techtarget.com/whatis/definition/bitwise) - *Explanation of how bitwise operators work and their common applications.*
- **(2)** [Brilliant Math & Science Wiki's "Secure Hash Algorithms"](https://brilliant.org/wiki/secure-hashing-algorithms/) - *Outline of how SHA functions use bitwise operations to secure data.*
- **(3)** ["The First 30 Years of Cryptographic Hash Functions and the NIST SHA-3 Competition"](https://link.springer.com/chapter/10.1007/978-3-642-11925-5_1) - *Conference Paper by Preneel, B outlining the evolution of Cryptographic Hash Functions*

**Task 1 Solution Sources**
- **(4)** [Python's Documentation on Built-in Types](https://docs.python.org/3/library/stdtypes.html) describes how to use bitwise operators to implement the methods required for task 1. By following the links in the solution you will find the relevant excerpt for each operator.
</div>


**1.1 Rotate the bits in a 32-bit unsigned integer to the left by `n` places**

- This solution first uses modulo (`n %= 32`) to normalize rotation counts greater than 32, because any rotation of 32 would result in no actual change.

- The bits are then shifted left using [Python's bitwise-left-shift operator](https://docs.python.org/3/library/stdtypes.html#bitwise-left-shift) (`x << n`). A mask is used (`0xFFFFFFFF`) to limit the value to 32 bits.

- [Python's bitwise-right-shift operation](https://docs.python.org/3/library/stdtypes.html#bitwise-right-shift) (`x >> (32- n)`) is used to shift the left-most bits that would overflow to the right-most position, mimicking rotation.

- The bits moved left are combined with the bits moved right using [Python's bitwise OR operator](https://docs.python.org/3/library/stdtypes.html) (`|`). As outlined in Python's documentation, it uses the formula   
`(a | b) = a + b - (a * b)` to ensure that any bits marked as 1 in either part of the rotation are preserved in the output.

In [2]:
def rotl(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the rotation
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

**1.2 Rotate the bits in a 32-bit unsigned integer to the right by `n` places**
- This solution is just an inverse of `rotl`

In [3]:
def rotr(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the right rotation
    return ((x >> n) & 0xFFFFFFFF) | (x << (32 - n))

**1.3 Choose the bits from `y` where `x` has bits set to `1` and bits in `z` where `x` has bits set to `0`**
- [Python's AND bitwise operator](https://docs.python.org/3/library/stdtypes.html) is used to return bits from `y` where `x` has a 1 in the same position (`x & y`)
- [Python's NOT bitwise operator](https://docs.python.org/3/library/stdtypes.html) is used with AND to return bits from `z` where `x` has a 0 in the same position. It achieves this by first inverting `x`'s bits, then performing the same AND operation (`z & ~x`)
- Python's OR bitwise operator is again used to combine both sets, providing the result.

In [4]:
def ch(x, y, z):
    return (y & x) | (z & ~x)

**1.4 Take a majority vote of the bits in `x`, `y`, and `z`**
- Each permutation of the input is checked for shared 1 bits using the AND operator (`x & y`)(`z & y`)(`z & y`)
- The OR operator is used to return the bits where at least 2 inputs had a 1 in that position

In [5]:
def maj(x, y, z):
    return (x & y) | (z & x) | (z & y)

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1A - "Verifying `rotl`"**  
**We will now test `rotl(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `5120` |
| `2153` `8` | `551168` |
</div>


In [10]:
print(f"rotl(20, 8) -> {rotl(20, 8)}")
print(f"rotl(2153, 8) -> {rotl(2153, 8)}")

rotl(20, 8) -> 5120
rotl(2153, 8) -> 551168


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1B - "Verifying `rotr`"**  
**We will now test `rotr(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `335544320` |
| `2153` `8` | `36121346056` |
</div>


In [11]:
print(f"rotr(20, 8) -> {rotr(20, 8)}")
print(f"rotr(2153, 8) -> {rotr(2153, 8)}")

rotr(20, 8) -> 335544320
rotr(2153, 8) -> 36121346056


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1C - "Verifying `ch`"**  
**We will now test `ch(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `34` |
</div>


In [12]:
print(f"ch(20, 2153, 54) -> {ch(20, 2153, 54)}")

ch(20, 2153, 54) -> 34


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1D - "Verifying `maj`"**  
**We will now test `maj(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `52` |
</div>


In [13]:
print(f"maj(20, 2153, 54) -> {maj(20, 2153, 54)}")

maj(20, 2153, 54) -> 52


### Task 2: Hash Functions


This task involves converting a string hashing function from C to Python. The original function comes from Brian Kernighan and Dennis Ritchie's book "The C Programming Language", and is a prominent example of polynomial rolling hashing [(5)](https://en.wikipedia.org/wiki/The_C_Programming_Language). These functions see frequent use in the implementation of hash tables, providing efficient data retrieval. Hash functions convert inputs of varied sizes such as strings into fixed-size numerical values. They should aim to minimize collisions while remaining computationally efficient. 

The value `31` is used as a multiplier because:
- It is a prime number, which improves distribution of hash numbers by minimizing patterns [(6)](https://arbitrary-but-fixed.net/2022/04/28/why-prime-numbers-for-hashing.html)
- It can be efficiently computed by using bitwise operations `x * 31` = `(x << 5) - x`

The value `101` is used as a modulo because:
- Prime modulos also improve distribution [(7)](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus)
- "The mathematical properties of prime numbers contribute to the difficulty of reversing the hash" [(7)](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus)

It should be noted that in Schölzel's comparison of various primes across different languages and datatypes, he found that "the prime 31 seems to be neither a particularly bad nor a particularly good candidate" [(6)](https://arbitrary-but-fixed.net/2022/04/28/why-prime-numbers-for-hashing.html). However, his tests were concerned only with collision avoidance, not computational efficiency.

During the testing for this solution, I compare its results against a non-prime hash function to demonstrate its advantages.

<div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 900px;">

**Task 2 Background Sources**
- **(5)** ["The C Programming Language" by Brian Kernighan and Dennis Ritchie](https://en.wikipedia.org/wiki/The_C_Programming_Language) - *Foundational book that introduced a polynominal hashing function that became heavily influenctial across computer science.*
- **(6)** ["Why 31? Explaining the use of prime numbers in Java hash functions" - Christopher Schölzel](https://arbitrary-but-fixed.net/2022/04/28/why-prime-numbers-for-hashing.html) - *Article exploring the historical use of 31 as a prime in hash functions*
- **(7)** ["Why should hash functions use a prime number modulus?" - Design Guru](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus) - *Explanation of the use 101 as a modulos in hash functions*

**Task 2 Solution Sources**
- **(8)** [Python's Documentation on `ord()`](https://docs.python.org/3/library/functions.html#ord) describes how `ord()` is used to return an integer representing the Unicode point of the input character.
</div>

**2.1 Generate a hash value for a string.**
- This solution iterates through the characters of a string. For each character, [Python's ord()](https://docs.python.org/3/library/functions.html#ord) is used to get its Unicode point.
- Each value is combined into a running total using `31` as a multiplier
- The result is then reduced using a modulo of `%101` to keep hash values within a range of 0 - 100

In [3]:
def hash_string(s: str) -> int:
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval
    return hashval % 101

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 350px;">
    
##### **Test 2A - "Verifying `hash_string`"**  
**We will now test `hash_string(s)` case sensitivity**

**Expected results:**

| Input  | Output |
|--------|--------|
| `Brutus` | `26` |
| `brutus` | `36` |
</div>


In [4]:
print(f"hash_string(Brutus) -> {hash_string('Brutus')}")
print(f"hash_string(brutus) -> {hash_string('brutus')}")

hash_string(Brutus) -> 26
hash_string(brutus) -> 36


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 700px;">
    
##### **Test 2B - "Testing `hash_string`"**  
**We will now test `hash_string(s)` use of modulo to normalize the hash to fall between 0-100**

**Expected results:**

| Input  | Output |
|--------|--------|
| `This is a demonstration of the modulo effect in hashing` | `<100` |
</div>

In [8]:
print(f"hash_string(This is a demonstration of the modulo effect in hashing) -> {hash_string('This is a demonstration of the modulo effect in hashing')}")

hash_string(This is a demonstration of the modulo effect in hashing) -> 50


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 700px;">
    
##### **Test 2C - "Testing `hash_string`"**  
**We will now compare `hash_string(s)` against a similar method that doesn't use prime numbers**

**Expected results:**

| Input  | Output |
|--------|--------|
| `A list of strings` | `hash_string` should outperform `non_prime_hash_string`|
</div>

In [18]:
def non_prime_hash_string(s: str) -> int:
    hashval = 0
    for char in s:
        hashval = ord(char) + 10 * hashval
    return hashval % 100

def compare_hash_functions():
    """Compare prime vs non-prime hash functions"""
    test_strings = [
        "by",         
        "cde"
    ]
    
    print("\nPrime-based hash function (31, 101):")
    prime_hashes = [hash_string(s) for s in test_strings]
    prime_unique = len(set(prime_hashes))
    
    print("\nNon-prime hash function (10, 100):")
    non_prime_hashes = [non_prime_hash_string(s) for s in test_strings]
    non_prime_unique = len(set(non_prime_hashes))
    
    # Print side by side comparison
    print("\nString    | Prime Hash | Non-Prime Hash")
    print("----------|------------|---------------")
    for i, s in enumerate(test_strings):
        print(f"{s:<10} | {prime_hashes[i]:<10} | {non_prime_hashes[i]:<10}")
    
    # Print collision stats
    print(f"\nPrime function unique values: {prime_unique}/{len(test_strings)}")
    print(f"Non-prime function unique values: {non_prime_unique}/{len(test_strings)}")
    
    if prime_unique > non_prime_unique:
        print("\nTest Passed: The prime hash function performed better.")
    else:
        print("\nTest Failed: The prime hash function did not perform better.")
# Run the test
compare_hash_functions()


Prime-based hash function (31, 101):

Non-prime hash function (10, 100):

String    | Prime Hash | Non-Prime Hash
----------|------------|---------------
by         | 28         | 1         
cde        | 67         | 1         

Prime function unique values: 2/2
Non-prime function unique values: 1/2

Test Passed: The prime hash function performed better.


### Task 3: SHA256


In [109]:
def calculateSHA256padding(file_path):
    # Get the file size in bits
    file_size_bits = os.path.getsize(file_path) * 8
    
    # Add 1 bit
    remainder = (file_size_bits + 1) % 512
    # If the message and padding fits, use one block
    if remainder <= 448:
        padding_bits = 448 - remainder
    # If the message and padding doesn't fit, use another block
    else:
        padding_bits = 960 - remainder
    
    # Create padding: 1 bit followed by zeros
    padding = b'\x80' + b'\x00' * (padding_bits // 8)
    
    # Append 64-bit representation of the original length
    length_bytes = struct.pack('>Q', file_size_bits)
    
    # Complete padding
    padding += length_bytes
    
    print(f"Paddings hex: {padding.hex()}")
    return padding.hex()

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 700px;">
    
##### **Test 3A - "Testing `calculateSHA256padding`"**  
**We will now test `calculateSHA256padding` for a file containing "abc"**

**Expected results:**

| Input  | Output |
|--------|--------|
| `abc` | `test_file_abc_padding` should return a total length of 512 bits|
| `abc` | `calculateSHA256padding` should return a hex of 80 00 00 00 00 00 ... 18|

</div>

In [ ]:
def test_abc_file_padding():
    with tempfile.NamedTemporaryFile(delete=False) as temp:
        temp.write(b'abc')
        temp_path = temp.name

    padding = bytes.fromhex(calculateSHA256padding(temp_path))
    os.remove(temp_path)

    total_length = len(padding) * 8
    correct_length = (len(padding) * 8) == 512
    correct_start = padding[0] == 0x80
    correct_end = padding[-8:] == struct.pack('>Q', 0)

    if correct_length and correct_start and correct_end:
        print(f"Test 3A passed: Total length is {total_length} bits")
    else:
        print("Test 3A failed: Total length is {total_length} bits")

test_empty_file_padding()

Paddings hex: 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Test 3A passed: Total length is 512 bits


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 750px;">
    
##### **Test 3B - "Testing `calculateSHA256padding`"**  
**We will now test `calculateSHA256padding` for a file with 56 bytes, requiring an extra block to fit padding**

**Expected results:**

| Input  | Output |
|--------|--------|
| `File with 56 bytes` | `calculateSHA256padding` should return a total length of 1024 bits|
</div>

In [108]:
def test_padding_causes_extra_block():
    # 56 bytes = 448 bits so any padding forces an extra block
    with tempfile.NamedTemporaryFile(delete=False) as temp:
        temp.write(b'A' * 56)
        temp_path = temp.name

    padding = bytes.fromhex(calculateSHA256padding(temp_path))

    # Get the file size in bits
    file_size_bits = os.path.getsize(temp_path) * 8  # File size in bits
    total_length = file_size_bits + len(padding) * 8

    correct_length = total_length == 1024  # Expect 1024 bits total
    correct_start = padding[0] == 0x80
    correct_end = padding[-8:] == struct.pack('>Q', 448)

    if correct_length and correct_start and correct_end:
        print(f"Test 3B passed: Total length is {total_length} bits, padding is correct")
    else:
        print(f"Test 3B failed: Total length is {total_length} bits")
        print(f"Padding bits: {len(padding) * 8}")
        print(f"Starts with 0x80? {padding[0] == 0x80}")
        print(f"Ends with length 448? {padding[-8:] == struct.pack('>Q', 448)}")

        os.remove(temp_path)
    

test_padding_causes_extra_block()


Test 3B passed: Total length is 1024 bits, padding is correct


<div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 390px;">

<b>FIPS PUB 180-4, Secure Hash Standard <a href="https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf" style="color:#1976D2; text-decoration: underline;"></b>(Link)</a>

This document outlines the official standards for SHA (Secure Hash Algorithms), which are widely used to ensure data integrity, secure passwords, and support digital signatures in modern cryptographic systems.
</div>


### Task 4: Prime Numbers

Approach A: Sieve of Eratosthenes

In [58]:
def sieve_of_eratosthenes(limit):
    primes = [True] * (limit + 1)
    primes[0] = primes[1] = False

    for i in range(2, int(limit**0.5) + 1):
        if primes[i]:
            for j in range(i * i, limit + 1, i):
                primes[j] = False

    return [num for num, is_prime in enumerate(primes) if is_prime]


Approach B: Sieve of Sundaram

In [60]:
def sieve_of_sundaram(limit):
    m = (limit - 1) // 2
    sieve = [False] * (m + 1)

    for i in range(1, m + 1):
        j = i
        while i + j + 2 * i * j <= m:
            sieve[i + j + 2 * i * j] = True
            j += 1

    primes = [2] 
    odd_primes = [2 * i + 1 for i in range(1, m + 1) if not sieve[i]]

    primes.extend(odd_primes)

    return [prime for prime in primes if prime <= limit]


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 600px;">
    
##### **Test 4A - "Verifying `sieve_of_sundaram` and `sieve_of_eratosthenes`"**  
We will now test both algorithms and compare their results

Expected results: Both algorithms should have identical outputs
</div>


In [61]:
primes_eratosthenes = sieve_of_eratosthenes(100)

primes_sundaram = sieve_of_sundaram(100)

if primes_eratosthenes == primes_sundaram:
    print("Test 4A Passed: Both methods have identical outputs.")
else:
    print("Test 4A Failed: The methods have different outputs.")

Test 4A Passed: Both methods have identical outputs.


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 700px;">
    
##### **Test 4B - "Comparing `sieve_of_sundaram` and `sieve_of_eratosthenes` runtime"**  
We will now time both algorithms runtime for returning the first 10,000 primes
</div>


In [62]:
import time

def test_performance():
    # Timing Eratosthenes sieve for limit 10,000
    start = time.time()
    sieve_of_eratosthenes(10000)
    end = time.time()
    print(f"Eratosthenes sieve (limit 10,000) took {end - start:.5f} seconds")

    # Timing Sundaram sieve for limit 10,000
    start = time.time()
    sieve_of_sundaram(10000)
    end = time.time()
    print(f"Sundaram sieve (limit 10,000) took {end - start:.5f} seconds")
    
test_performance()

Eratosthenes sieve (limit 10,000) took 0.00103 seconds
Sundaram sieve (limit 10,000) took 0.00300 seconds


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 600px;">
    
##### **Test 4C - "Comparing `sieve_of_sundaram` and `sieve_of_eratosthenes` accuracy"**  
We will now if a selection of primes appear in both algorithms
</div>

In [63]:
def checking_primes():
    random_primes = [7417, 3709, 547, 2383, 809, 13]
    primes_sundaram = sieve_of_sundaram(10000)
    primes_eratosthenes = sieve_of_eratosthenes(10000)

    all_found_in_sundaram = all(prime in primes_sundaram for prime in random_primes)
    all_found_in_eratosthenes = all(prime in primes_eratosthenes for prime in random_primes)

    if all_found_in_sundaram:
        print("Test 4C Passed: All random primes found in Sundaram sieve.")
    else:
        print("Test 4C Failed: Not all random primes found in Sundaram sieve.")
    
    if all_found_in_eratosthenes:
        print("Test 4C Passed: All random primes found in Eratosthenes sieve.")
    else:
        print("Test 4C Failed: Not all random primes found in Eratosthenes sieve.")

checking_primes()

Test 4C Passed: All random primes found in Sundaram sieve.
Test 4C Passed: All random primes found in Eratosthenes sieve.


<div style="display: flex; gap: 20px;">

  <div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 330px;">
    <b><a href="https://www.geeksforgeeks.org/sieve-of-eratosthenes/" style="color:#1976D2; text-decoration: underline;">Sieve of Eratosthenes</b></a><br><br>
    This page explains the Sieve of Eratosthenes algorithm for efficiently generating all prime numbers less than a given number, an important method in number theory and cryptography.
  </div>

  <div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 330px;">
    <a href="https://www.geeksforgeeks.org/sieve-sundaram-print-primes-smaller-n/" style="color:#1976D2; text-decoration: underline;"><b>Sieve of Sundaram</b></a><br><br>
    This page explains the Sieve of Sundaram algorithm for finding prime numbers smaller than a given number, a method often used in number theory and computational applications.
  </div>

</div>

### Task 5: Roots

#### 1.1 First 100 Prime Numbers

Before calculating the square root of the first 100 prime numbers, we need those numbers.

To get the first 100 prime numbers, we can use the `sieve_of_eratosthenes` method from the previous task

In [64]:
first_100_primes = sieve_of_eratosthenes(550)

#### 1.2 Square Root of Each Prime Number

In [65]:
import math

square_roots = [math.sqrt(p) for p in first_100_primes]

#### 1.3 Extract Fractional Part of Square Roots

In [66]:
fractional_parts = [sqrt - int(sqrt) for sqrt in square_roots]

#### 1.4 Calculate First 32 Bits from Each Square Root

In [76]:
def get_first_32_bits(fractional_part):
    bits = int(fractional_part * (2**32))

    return format(bits, '032b')  

binary_fractions = [get_first_32_bits(fraction) for fraction in fractional_parts]

#### 1.5 Print Results

In [77]:
for i, binary_fraction in enumerate(binary_fractions):
    print(f"{first_100_primes[i]:>3} -> {binary_fraction}")

  2 -> 01101010000010011110011001100111
  3 -> 10111011011001111010111010000101
  5 -> 00111100011011101111001101110010
  7 -> 10100101010011111111010100111010
 11 -> 01010001000011100101001001111111
 13 -> 10011011000001010110100010001100
 17 -> 00011111100000111101100110101011
 19 -> 01011011111000001100110100011001
 23 -> 11001011101110111001110101011101
 29 -> 01100010100110100010100100101010
 31 -> 10010001010110010000000101011010
 37 -> 00010101001011111110110011011000
 41 -> 01100111001100110010011001100111
 43 -> 10001110101101000100101010000111
 47 -> 11011011000011000010111000001101
 53 -> 01000111101101010100100000011101
 59 -> 10101110010111111001000101010110
 61 -> 11001111011011001000010111010011
 67 -> 00101111011100110100011101111101
 71 -> 01101101000110000010011011001010
 73 -> 10001011010000111101010001010111
 79 -> 11100011011000001011010110010110
 83 -> 00011100010001010110000000000010
 89 -> 01101111000110010110001100110001
 97 -> 11011001010011101011111010110001


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 750px;">
    
##### **Test 5A - "Verifying `binary_fraction`**  
We will now test `binary_fractions` calculation against the first 32 bits of the fractional part of 72's square root

72 - 8.48528137424... - 48528137424 - 01111100001110110110011001101111
</div>

In [78]:
square_root_72 = math.sqrt(72)

fractional_part_72 = square_root_72 - int(square_root_72)

first_32_bits_72 = get_first_32_bits(fractional_part_72)

print(f"First 32 bits of fractional part of square root of 72: {first_32_bits_72}")

if first_32_bits_72 == "01111100001110110110011001101111":
    print("Test 5A Passed: The first 32 bits match.")
else:
    print("Test 5A Failed: The first 32 bits do not match.")

First 32 bits of fractional part of square root of 72: 01111100001110110110011001101111
Test 5A Passed: The first 32 bits match.


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 750px;">
    
##### **Test 5B - "Verifying `binary_fraction`**  
We will now test `binary_fractions` against 8 known hex constants of the fractional part of the first 8 prime numbers

We will be converting our `binary_fractions` to hex for comparison.

These hex constants are used in SHA-256 as the initial hash values
</div>

In [82]:
def test_binary_fractions_against_sha_constants():
    expected_hex = [
        "6a09e667",
        "bb67ae85",
        "3c6ef372",
        "a54ff53a",
        "510e527f",
        "9b05688c",
        "1f83d9ab",
        "5be0cd19"
    ]

    all_passed = True

    for i, expected in enumerate(expected_hex):
        # Convert binary string to hex
        actual_bin = binary_fractions[i]
        actual_hex = format(int(actual_bin, 2), '08x')

        if actual_hex == expected:
            print(f"PASS: Prime {first_100_primes[i]} → {actual_hex}")
        else:
            print(f"FAIL: Prime {first_100_primes[i]} → got {actual_hex}, expected {expected}")
            all_passed = False

    if all_passed:
        print("Test 5B Passed: Top 8 binary fractions match known SHA-256 constants.")
    else:
        print("Test 5B Failed: One or more values did not match.")

test_binary_fractions_against_sha_constants()        


PASS: Prime 2 → 6a09e667
PASS: Prime 3 → bb67ae85
PASS: Prime 5 → 3c6ef372
PASS: Prime 7 → a54ff53a
PASS: Prime 11 → 510e527f
PASS: Prime 13 → 9b05688c
PASS: Prime 17 → 1f83d9ab
PASS: Prime 19 → 5be0cd19
Test 5B Passed: Top 8 binary fractions match known SHA-256 constants.


### Task 6: Proof of Work

In [87]:
import hashlib

def count_leading_zero_bits(digest):
    count = 0
    for byte in digest:
        for i in range(8):
            if (byte >> (7 - i)) & 1 == 0:
                count += 1
            else:
                return count
    return count


In [88]:
def load_words(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f if line.strip().isalpha()]

def find_best_words(words):
    max_zeros = 0
    best_words = []

    for word in words:
        digest = hashlib.sha256(word.encode("utf-8")).digest()
        zeros = count_leading_zero_bits(digest)

        if zeros > max_zeros:
            max_zeros = zeros
            best_words = [word]
        elif zeros == max_zeros:
            best_words.append(word)

    return max_zeros, best_words

In [89]:
word_file = "dictionary.txt"  
word_list = load_words(word_file)

max_bits, top_words = find_best_words(word_list)

print(f"Max leading zero bits: {max_bits}")
print("Top word(s):")
for word in top_words:
    print(word)

Max leading zero bits: 18
Top word(s):
goaltenders


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 750px;">
    
##### **Test 6A - "Verifying `find_best_words`**  
We will now test `find_best_words` with a list of 5 words, from which we know "flight" has the highest leading zero bits
</div>

In [94]:
def test_flight():
    words = ["cold", "empire", "flight", "America", "holiday"]
    max_zeros, top_words = find_best_words(words)
    print(f"Max leading zero bits: {max_zeros}")
    print(f"Top word(s): {top_words}")
    if max_zeros == 6 and top_words == ["flight"]:
        print("Test 6A passed: The word 'flight' has the most leading zero bits.")
    else:
        print("Test 6A failed: The word 'flight' was not selected.")

test_flight()

Max leading zero bits: 6
Top word(s): ['flight']
Test 6A passed: The word 'flight' has the most leading zero bits.


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 750px;">
    
##### **Test 6B - "Verifying `find_best_words`**  
We will now test `find_best_words` with a list of 5 words, from which we know the words "apple" and "cherry" are tied for the most leading 0 bits
</div>

In [96]:
def test_tie():
    words = ["apple", "banana", "cherry", "a", "elderberry"]
    max_zeros, top_words = find_best_words(words)
    print(f"Max leading zero bits: {max_zeros}")
    print(f"Top word(s): {top_words}")
    if len(top_words) > 1:
        print("Test 6B passed: There was a tie.")
    else:
        print("Test 6B failed: There was no tie.")

test_tie()

Max leading zero bits: 2
Top word(s): ['apple', 'cherry']
Test 6B passed: There was a tie.


### Task 7: Turing Machines

In [29]:
# Default Turing machine configuration
tape = []
head = 0
state = 'q0'

In [30]:
# Define transition rules
def get_transitions():
    return {
        ('q0', '0'): ('q0', '0', 'R'),
        ('q0', '1'): ('q0', '1', 'R'),
        ('q0', '_'): ('q1', '_', 'L'),
        
        ('q1', '0'): ('qf', '1', 'N'),
        ('q1', '1'): ('q2', '0', 'L'),

        ('q2', '0'): ('qf', '1', 'N'),
        ('q2', '1'): ('q2', '0', 'L'),
        ('q2', '_'): ('qf', '1', 'N')
    }

In [31]:
# Step function for the Turing machine
def step():
    global head, state, tape
    symbol = tape[head] if 0 <= head < len(tape) else '_'
    key = (state, symbol)
    
    transitions = get_transitions()
    
    if key not in transitions:
        return
    
    new_state, new_symbol, direction = transitions[key]
    
    tape[head] = new_symbol
    state = new_state

    if direction == 'R':
        head += 1
    elif direction == 'L':
        head -= 1

    # Tape extension
    if head < 0:
        tape.insert(0, '_')
        head = 0
    elif head >= len(tape):
        tape.append('_')


In [ ]:
def run_turing_machine(input_binary):
    global tape, head, state
    # Initialize tape with input binary, return 1 if input is empty
    if input_binary == "":
        return "1"
    else:
        tape = list(input_binary) + ['_']
    
    head = 0
    state = 'q0'
    
    steps = 0
    while state != 'qf':
        step()
        steps += 1
    
    return ''.join(tape).strip('_')

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 350px;">
    
##### **Test 7A - Testing `add_one_binary` No Carry**  
**We will now verify our turing machine correctly handles a case when no carry is required**

**Expected results:**

| Input  | Output |
|--------|--------|
| `10010` | `10011` |
</div>

In [33]:
# Test 7A - No Carry
result = run_turing_machine("10010")
print(result)
if result == "10011":
    print(f"Test 7A passed!")
else:
    print(f"Test 7A failed. Expected: 10011")

10011
Test 7A passed!


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 400px;">
    
##### **Test 7B - "Testing Binary Increment: Single Carry"**  
**We will verify our Turing machine correctly handles a case requiring a single carry operation when the least significant bit is 1.**

**Expected results:**

| Input  | Output |
|--------|--------|
| `101` | `110` |
</div>

In [34]:
# Test 7B -  Carry
result = run_turing_machine("101")
print(result)
if result == "110":
    print("Test 7B passed!")
else:
    print(f"Test 7B failed. Expected: 110")

110
Test 7B passed!


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 400px;">
    
##### **Test 7C - "Testing Binary Increment: Multiple Carries"**  
**We will verify our Turing machine correctly handles a situation requiring multiple carry operations when consecutive least significant bits are 1.**

**Expected results:**

| Input  | Output |
|--------|--------|
| `1111` | `10000` |
</div>

In [35]:
# Test 7C - Multiple Carries
result = run_turing_machine("1111")
print(result)
if result == "10000":
    print("Test 7C passed!")
else:
    print(f"Test 7C failed. Expected: 10000")

10000
Test 7C passed!


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 500px;">
    
##### **Test 7D - "Testing Binary Increment: Empty String Edge Case"**  
**We will test our Turing machine's ability to handle an empty input, which should be interpreted as 0.**

**Expected results:**

| Input  | Output |
|--------|--------|
| `""` | `"1"` |
</div>

In [40]:
# Test 7D - Empty String
result = run_turing_machine("")
print(result)
if result == "1":
    print("Test 7D passed!")
else:
    print(f"Test 7D failed. Expected: 1")

1
Test 7D passed!


### Task 8: Computational Complexity

In [98]:
def bubble_sort(arr):
    # Copy the array to avoid modifying the original
    arr = arr.copy()
    # set the comparison count to 0
    comparison_count = 0
    # Get the length of the array
    n = len(arr)
    
    # Move through the array
    for i in range(n):
        # Flag to check if a swap occurred
        swapped = False
        
        # Exclude the last i numbers as they are already sorted
        for j in range(0, n-i-1):
            # Increment the comparison count
            comparison_count += 1
            # Swap if the number found is greater than the next number
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True
        # If no two elements were swapped in the inner loop, then break
        if swapped == False:
            break
    
    return arr, comparison_count

bubble_sorted = bubble_sort([1, 2, 3, 4, 5])
print("Bubble Sorted Array:", bubble_sorted[0], "Comparison Count:", bubble_sorted[1])

Bubble Sorted Array: [1, 2, 3, 4, 5] Comparison Count: 4


In [99]:
import itertools

def get_all_permutations(arr):
    # Generate all permutations using itertools
    all_permutations = list(itertools.permutations(arr))
    
    # Convert each permutation tuple to a list
    result = [list(perm) for perm in all_permutations]
    
    return result

In [100]:
all_permutations = get_all_permutations([1, 2, 3, 4, 5])

for i, perm in enumerate(all_permutations):
    bubble_sorted = bubble_sort(perm)
    print(f"Permutation {i+1}: {perm} -> Sorted: {bubble_sorted[0]}, Comparison Count: {bubble_sorted[1]}")

Permutation 1: [1, 2, 3, 4, 5] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 4
Permutation 2: [1, 2, 3, 5, 4] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 7
Permutation 3: [1, 2, 4, 3, 5] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 7
Permutation 4: [1, 2, 4, 5, 3] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 9
Permutation 5: [1, 2, 5, 3, 4] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 7
Permutation 6: [1, 2, 5, 4, 3] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 9
Permutation 7: [1, 3, 2, 4, 5] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 7
Permutation 8: [1, 3, 2, 5, 4] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 7
Permutation 9: [1, 3, 4, 2, 5] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 9
Permutation 10: [1, 3, 4, 5, 2] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 10
Permutation 11: [1, 3, 5, 2, 4] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 9
Permutation 12: [1, 3, 5, 4, 2] -> Sorted: [1, 2, 3, 4, 5], Comparison Count: 10
Permutation 13: [1, 4, 2, 3, 5] -> Sorted: [1, 

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 500px;">
    
##### **Test 8A - "Testing `bubble_sort`: Already Sorted List"**  
**We will test the comparisons needed to sort an already sorted list**

**Expected results:**

| Input           | Output           |
|-----------------|------------------|
| `[1, 2, 3, 4, 5]` | `Comparisons Count - 4` |

</div>

In [104]:
arr = [1, 2, 3, 4, 5]
bubble_sorted = bubble_sort(arr)
print("Bubble Sorted Array:", bubble_sorted[0], "Comparison Count:", bubble_sorted[1])

Bubble Sorted Array: [1, 2, 3, 4, 5] Comparison Count: 4


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 500px;">
    
##### **Test 8B - "Testing `bubble_sort`: Reverse Sorted List"**  
**We will test the comparisons needed to sort a list that is in reverse order**

**Expected results:**

| Input           | Output           |
|-----------------|------------------|
| `[5, 4, 3, 2, 1]` | `Comparison Count - 10` |

</div>

In [102]:
arr = [5, 4, 3, 2, 1]
bubble_sorted = bubble_sort(arr)
print("Bubble Sorted Array:", bubble_sorted[0], "Comparison Count:", bubble_sorted[1])

Bubble Sorted Array: [1, 2, 3, 4, 5] Comparison Count: 10


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 500px;">
    
##### **Test 8C - "Testing `bubble_sort`: List with Identical Elements"**  
**We will test the comparisons needed to sort a list filled with the same number.**

**Expected results:**

| Input           | Output           |
|-----------------|------------------|
| `[3, 3, 3, 3, 3]` | `Comparison Count - 4` |

</div>

In [105]:
arr = [3, 3, 3, 3, 3]
bubble_sorted = bubble_sort(arr)
print("Bubble Sorted Array:", bubble_sorted[0], "Comparison Count:", bubble_sorted[1])

Bubble Sorted Array: [3, 3, 3, 3, 3] Comparison Count: 4
